<a href="https://colab.research.google.com/github/nevetto/FakeNewsBidirectional/blob/main/FakeNewsBidirectional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fake News Classifier Using Biderectional

In [ ]:
# Dataset:  https://www.kaggle.com/competitions/fake-news/data?select=train.csv

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

In [5]:
df= pd.read_csv('train.csv')

In [6]:
# df= pd.read_csv('train.csv', engine='python',error_bad_lines=False)

In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
df.shape

(20800, 5)

In [10]:
df = df.dropna()

In [11]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [12]:
# get the independent features
X = df.drop('label', axis=1)

In [13]:
y = df['label']

In [15]:
# check whether dataset is balanced or not
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [16]:
import tensorflow as tf


In [17]:
tf.__version__

'2.8.2'

In [19]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [20]:
# vocabulary size
voc_size = 5000

In [21]:
messages = X.copy()

In [22]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [24]:
import nltk
import re
from nltk.corpus import stopwords

In [25]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [27]:
messages.reset_index(inplace=True)

In [28]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [29]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [30]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
print(onehot_repr)

[[2872, 2592, 2925, 2902, 3638, 4063, 3197, 1815, 1033, 2852], [41, 1331, 1961, 332, 3529, 1889, 90], [1229, 134, 4634, 3435], [4224, 260, 565, 2990, 1633, 210], [731, 3529, 3318, 441, 510, 2445, 3529, 2833, 383, 2252], [2711, 2359, 2611, 1456, 3808, 3335, 3141, 2123, 1282, 1254, 2878, 4535, 4001, 1072, 90], [3002, 3871, 4985, 4841, 1021, 1797, 1510, 2332, 1314, 1300, 1217], [2032, 4638, 2226, 1918, 3506, 3245, 3335, 210, 1314, 1300, 1217], [4064, 1461, 1614, 3628, 4013, 2360, 22, 1116, 3335, 4079], [1401, 442, 2946, 2325, 4007, 4666, 2446, 2137], [4360, 2481, 3651, 1446, 1078, 3762, 1854, 595, 4952, 1640, 1300], [2990, 4348, 3638, 2360, 3335, 3506], [2279, 1856, 937, 2374, 4506, 3671, 1013, 934, 4532], [4118, 3553, 2328, 4991, 1343, 2172, 4438, 1314, 1300, 1217], [3115, 2870, 3723, 2338, 2850, 1314, 1300, 1217], [2089, 3563, 967, 1256, 3283, 2031, 463, 1715, 3343, 3864], [2511, 1709, 1331], [1366, 2472, 1170, 2712, 3335, 430, 4874, 90], [3971, 77, 1961, 3187, 1386, 2751, 338, 4992, 24

# Embedding Representation

In [31]:
sent_length=20
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 1815, 1033, 2852],
       [   0,    0,    0, ..., 3529, 1889,   90],
       [   0,    0,    0, ...,  134, 4634, 3435],
       ...,
       [   0,    0,    0, ..., 1314, 1300, 1217],
       [   0,    0,    0, ..., 2345, 2070,  878],
       [   0,    0,    0, ..., 3277, 3828,  571]], dtype=int32)

In [32]:
# Creating Model
embeding_vector_feature=40
model=Sequential()
model.add(Embedding(voc_size,embeding_vector_feature,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [33]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [34]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
# train test split

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [36]:
# Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test), epochs=10,batch_size=32)

Epoch 1/10
383/383 [==============================] - 12s 10ms/step - loss: 0.2722 - accuracy: 0.8729 - val_loss: 0.1969 - val_accuracy: 0.9127
Epoch 2/10
383/383 [==============================] - 4s 9ms/step - loss: 0.1268 - accuracy: 0.9512 - val_loss: 0.2164 - val_accuracy: 0.9198
Epoch 3/10
383/383 [==============================] - 3s 9ms/step - loss: 0.0861 - accuracy: 0.9693 - val_loss: 0.2194 - val_accuracy: 0.9147
Epoch 4/10
383/383 [==============================] - 3s 8ms/step - loss: 0.0522 - accuracy: 0.9824 - val_loss: 0.2576 - val_accuracy: 0.9147
Epoch 5/10
383/383 [==============================] - 3s 8ms/step - loss: 0.0295 - accuracy: 0.9918 - val_loss: 0.3925 - val_accuracy: 0.9138
Epoch 6/10
383/383 [==============================] - 3s 9ms/step - loss: 0.0168 - accuracy: 0.9956 - val_loss: 0.3966 - val_accuracy: 0.9128
Epoch 7/10
383/383 [==============================] - 3s 8ms/step - loss: 0.0130 - accuracy: 0.9961 - val_loss: 0.4246 - val_accuracy: 0.9122
Epoc

In [37]:
y_pred=model.predict(X_test)

In [38]:
y_pred>=0.5

array([[ True],
       [False],
       [False],
       ...,
       [False],
       [ True],
       [ True]])

In [40]:
import numpy as np
y_pred= np.where(y_pred>=0.5,1,0)

In [41]:
y_pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [43]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(y_test,y_pred)

array([[3054,  365],
       [ 153, 2463]])

In [44]:
print(accuracy_score(y_test,y_pred))

0.9141673570836786
